In [ ]:
import tensorflow as tf
tf.compat.v1.ConfigProto()
config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))

gpu_options=tf.compat.v1.GPUOptions(allow_growth=True) 
sess = tf.compat.v1.Session(config=config)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import keras
import random
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, Flatten
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from keras.layers import AveragePooling1D, MaxPooling1D, Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D,concatenate
from keras.layers import SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import Callback
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt

import csv
import os
import time
from math import floor
import warnings
from keras import backend as K
warnings.filterwarnings('ignore')
np.random.seed(42)
random.seed(12345)


In [ ]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
    print("TensorFlow version:", tf.__version__)
    gpus = tf.config.list_physical_devices('GPU')
    print("Number of available GPUs:", len(gpus))
    for i in range(len(gpus)):
        print("GPU name:", gpus[i].name)
else:
    print("GPU is not available.")

In [ ]:
with open('....pkl', 'rb') as f:
    Data = pickle.load(f)

In [ ]:
#Downsize
import numpy as np
x_original = Data.iloc[:, 1:].values
step_size = x_original.shape[1] / 2251
new_indices = (np.arange(0, x_original.shape[1], step_size)).astype(int)
x_downsampled = x_original[:, new_indices]
Data = pd.concat([Data.iloc[:, 0], pd.DataFrame(x_downsampled)], axis=1)

In [ ]:
iput_class = 3753
oput_class = 3753

In [ ]:
aug_time=50
numtrain = iput_class * aug_time
numval = int(0.2 * iput_class)
# Extract features and labels
X = Data.iloc[:, 1:].values
y = Data.iloc[:, 0].values
unique_labels = np.unique(y)
print(f'There are {len(unique_labels)} unique labels.')
# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split your data into training and validation sets:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert these to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.int32)

In [ ]:
X_train.shape

In [ ]:
keras.backend.clear_session()
# Construct CNN layers
# more para-LeNet-5
cnn_model = Sequential()
cnn_model.add(Conv1D(6, 5, strides = 1, activation = 'relu', input_shape = (2251, 1)))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(16, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(32, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
cnn_model.add(Conv1D(64, 5, strides = 1, activation = 'relu'))
cnn_model.add(MaxPooling1D(2, strides = 2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())
cnn_model.add(Dense(120, activation = 'relu'))
cnn_model.add(Dense(84, activation = 'relu'))
cnn_model.add(Dense(oput_class, activation = 'softmax', activity_regularizer = keras.regularizers.l2(0.1)))
cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
def log_to_csv(log_file,early_stopping_setting, epoch, elapsed_time, top1, top3, top5):
    # Check if file exists
    if not os.path.isfile(log_file):
        # Open file in write mode and write headers
        with open(log_file, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow(["early_stopping_setting","Epoch", "Elapsed_Time", "Top1", "Top3", "Top5"])
            
    # File exists, open in append mode and write data
    with open(log_file, mode='a') as f:
        writer = csv.writer(f)
        writer.writerow([early_stopping_setting,epoch, elapsed_time, top1, top3, top5])


In [ ]:
# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train.numpy())
y_val_onehot = to_categorical(y_val.numpy())

# Set up early stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
class CustomEarlyStopping(Callback):
    def __init__(self, patience=0, early_step_setting=0.95):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.early_step_setting = early_step_setting
        self.wait = 0
        self.stopped_epoch = 0
        self.best = 0
        self.losses = []

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current_loss = logs.get('loss')
        self.losses.append(current_loss)

        if epoch > 5 and current_loss > self.losses[epoch - 1] * self.early_step_setting:
            self.wait += 1
            print('\n',self.wait)
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Early stopping!")
early_stopping_setting=0.9
custom_early_stopping = CustomEarlyStopping(patience=3, early_step_setting=early_stopping_setting)
start_time = time.time()
cnn_model.fit(
    X_train.numpy().reshape(X_train.shape[0], X_train.shape[1], 1), 
    y_train_onehot, 
    batch_size = 128, 
    epochs = 20, 
    verbose = 1,
    validation_data=(X_val.numpy().reshape(X_val.shape[0], X_val.shape[1], 1), y_val_onehot),
    callbacks=[custom_early_stopping]
)
elapsed_time = time.time() - start_time
'''
    # Fit the CNN model
cnn_model.fit(
    X_train.numpy().reshape(X_train.shape[0], X_train.shape[1], 1), 
    y_train_onehot, 
    batch_size = 128, 
    epochs = 20, 
    verbose = 1,
    validation_data=(X_val.numpy().reshape(X_val.shape[0], X_val.shape[1], 1), y_val_onehot),
    callbacks=[early_stopping]
)
'''
    
def calculate_top_k_accuracy(y_true, y_pred, k):
    top_k_preds = np.argsort(y_pred, axis=-1)[:, -k:]
    target_in_top_k = [(true in top_k) for true, top_k in zip(y_true, top_k_preds)]
    return np.mean(target_in_top_k)

# Make predictions
y_pred = cnn_model.predict(X_val.numpy().reshape(X_val.shape[0], X_val.shape[1], 1))

# Calculate top-1, top-3, top-5 accuracies
top_1_accuracy = calculate_top_k_accuracy(y_val.numpy(), y_pred, 1)
top_3_accuracy = calculate_top_k_accuracy(y_val.numpy(), y_pred, 3)
top_5_accuracy = calculate_top_k_accuracy(y_val.numpy(), y_pred, 5)

print(f"Top-1 Accuracy: {round(top_1_accuracy, 2) * 100}%")
print(f"Top-3 Accuracy: {round(top_3_accuracy, 2) * 100}%")
print(f"Top-5 Accuracy: {round(top_5_accuracy, 2) * 100}%")

# Plot training loss
plt.plot(custom_early_stopping.losses)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()

In [ ]:
from datetime import datetime
top_1_accuracies = []
top_3_accuracies = []
top_5_accuracies = []
total_times = []
num_classes=3753
for iiiii in range(10):
    keras.backend.clear_session()
    # Construct CNN layers
    # more para-LeNet-5
    pretrained_model = keras.models.load_model('CNNmodel_test_top1_accuracy=0.97_test_top3_accuracy=1.0_test_top5_accuracy=1.0_test_current_time=20230619-224155.h5')

    # Remove the last layer
    pretrained_model._layers.pop()

    cnn_model = keras.models.Sequential()
    for layer in pretrained_model.layers[:-1]:  # just exclude last layer from copying
        cnn_model.add(layer)

    # Add a new output layer to the new model
    new_output_layer = keras.layers.Dense(num_classes, activation='softmax', name='unique_dense_layer')
    cnn_model.add(new_output_layer)
    cnn_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    

    y_train_onehot = to_categorical(y_train)
    y_val_onehot = to_categorical(y_val)

    # Set up early stopping
    #early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)


    early_stopping_setting=0.98
    custom_early_stopping = CustomEarlyStopping(patience=3, early_step_setting=early_stopping_setting)
    start_time = time.time()
    cnn_model.fit(
    X_train.numpy().reshape(X_train.shape[0], X_train.shape[1], 1), 
    y_train_onehot, 
    batch_size = 128, 
    epochs = 50, 
    verbose = 1,
    validation_data=(X_val.numpy().reshape(X_val.shape[0], X_val.shape[1], 1), y_val_onehot),
    callbacks=[custom_early_stopping]
    )
    elapsed_time = time.time() - start_time
    
    with open('FTIR_test_data.pkl', 'rb') as f:
        Test = pickle.load(f)
    X_test = Test.iloc[:, 1:].values
    X_test.shape
    X_test_resampled = X_test[:, new_indices]
    X_test_resampled.shape
    df_resampled = pd.DataFrame(X_test_resampled, index=Test.index)
    X_test = df_resampled.values
    y_true = Test.iloc[:,0]
    unique_labels_test = np.unique(y_true)
    missing_labels = set(unique_labels_test).difference(set(unique_labels))
    y_true_series = pd.Series(y_true)
    mask = ~y_true_series.isin(missing_labels)
    X_test = X_test[mask.values]
    y_true = y_true_series[mask].values
    # Make predictions
    y_pred = cnn_model.predict(X_test.reshape(len(y_true), 2251, 1))

    # Get the top k predictions
    top_1_preds = np.argsort(y_pred, axis=-1)[:, -1:]
    top_3_preds = np.argsort(y_pred, axis=-1)[:, -3:]
    top_5_preds = np.argsort(y_pred, axis=-1)[:, -5:]

    # Get the true labels in encoded format
    true_labels_encoded = label_encoder.transform(y_true)


    # Initialize counters
    correct_top1 = 0
    correct_top3 = 0
    correct_top5 = 0
    total = len(true_labels_encoded)

    # First loop to calculate accuracy
    for i, true_label_encoded in enumerate(true_labels_encoded):
        true_label = label_encoder.inverse_transform([true_label_encoded])[0]

        # Decode the predicted labels
        predicted_label_top1 = label_encoder.inverse_transform(top_1_preds[i])[0]
        predicted_label_top3 = label_encoder.inverse_transform(top_3_preds[i])
        predicted_label_top5 = label_encoder.inverse_transform(top_5_preds[i])

        if true_label == predicted_label_top1:
            correct_top1 += 1

        if true_label in predicted_label_top3:
            correct_top3 += 1

        if true_label in predicted_label_top5:
            correct_top5 += 1

    # Print accuracy
    print(f'Top-1 Accuracy: {correct_top1 / total * 100}%')
    print(f'Top-3 Accuracy: {correct_top3 / total * 100}%')
    print(f'Top-5 Accuracy: {correct_top5 / total * 100}%')
    # Append the results
    top_1_accuracies.append(correct_top1 / total * 100)
    top_3_accuracies.append(correct_top3 / total * 100)
    top_5_accuracies.append(correct_top5 / total * 100)
    total_times.append(elapsed_time)
# Calculate mean and std for accuracies and elapsed time
mean_top_1_accuracy = np.mean(top_1_accuracies)
std_top_1_accuracy = np.std(top_1_accuracies)

mean_top_3_accuracy = np.mean(top_3_accuracies)
std_top_3_accuracy = np.std(top_3_accuracies)

mean_top_5_accuracy = np.mean(top_5_accuracies)
std_top_5_accuracy = np.std(top_5_accuracies)

mean_time = np.mean(total_times)
std_time = np.std(total_times)

# Create a dataframe
results_df = pd.DataFrame({
    'Top_1_Accuracy': top_1_accuracies,
    'Top_3_Accuracy': top_3_accuracies,
    'Top_5_Accuracy': top_5_accuracies,
    'Elapsed_Time': total_times
})

# Add mean and std rows
results_df = results_df.append({
    'Top_1_Accuracy': mean_top_1_accuracy,
    'Top_3_Accuracy': mean_top_3_accuracy,
    'Top_5_Accuracy': mean_top_5_accuracy,
    'Elapsed_Time': mean_time
}, ignore_index=True)

results_df = results_df.append({
    'Top_1_Accuracy': std_top_1_accuracy,
    'Top_3_Accuracy': std_top_3_accuracy,
    'Top_5_Accuracy': std_top_5_accuracy,
    'Elapsed_Time': std_time
}, ignore_index=True)

# Save the dataframe to a csv file
results_df.to_csv('....csv', index=False)